Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/track-and-monitor-experiments/using-mlflow/train-remote/train-remote.png)

## Use MLflow with Azure Machine Learning for Remote Training Run

This example shows you how to use MLflow tracking APIs together with Azure Machine Learning services for storing your metrics and artifacts, from local Notebook run. You'll learn how to:

 1. Set up MLflow tracking URI so as to use Azure ML
 2. Create experiment
 3. Train a model on Machine Learning Compute while logging metrics and artifacts
 4. View your experiment within your Azure ML Workspace in Azure Portal.

## Prerequisites

Make sure you have completed the [Configuration](../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

## Set-up

Check Azure ML SDK version installed on your computer, and then connect to your Workspace.

In [1]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace, Experiment

print("SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()

SDK version: 1.40.0


Let's also create a Machine Learning Compute cluster for submitting the remote run. 

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=2)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpu-cluster


### Create Azure ML Experiment

The following steps show how to submit a training Python script to a cluster as an Azure ML run, while logging happens through MLflow APIs to your Azure ML Workspace. Let's first create an experiment to hold the training runs.

In [3]:
from azureml.core import Experiment

experiment_name = "RemoteTrain-with-mlflow-sample"
exp = Experiment(workspace=ws, name=experiment_name)

### Instrument remote training script using MLflow

Let's use [*train_diabetes.py*](train_diabetes.py) to train a regression model against diabetes dataset as the example. Note that the training script uses mlflow.start_run() to start logging, and then logs metrics, saves the trained scikit-learn model, and saves a plot as an artifact.

Run following command to view the script file. Notice the mlflow logging statements, and also notice that the script doesn't have explicit dependencies on azureml library.

In [4]:
training_script = 'train_diabetes.py'
with open(training_script, 'r') as f:
    print(f.read())

# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

import matplotlib
matplotlib.use('Agg')

with mlflow.start_run():
    X, y = load_diabetes(return_X_y=True)
    columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    data = {
        "train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

    mlflow.log_metric("Training samples", len(data['train']['X']))
    mlflow.log_metric("Test samples", len(data['test']['X']))

    # Log the algorithm parameter alpha to the run
    mlflow.log_metric('alpha', 0.03)
    # Create, fit, and test the scikit-lea

### Submit Run to Cluster 

Let's submit the run to cluster. When running on the remote cluster as submitted run, Azure ML sets the MLflow tracking URI to point to your Azure ML Workspace, so that the metrics and artifacts are automatically logged there.

Note that you have to specify the packages your script depends on, including *azureml-mlflow* that implicitly enables the MLflow logging to Azure ML. 

First, create a environment with Docker enable and required package dependencies specified.

In [5]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name="mlflow-env")

# Specify conda dependencies with scikit-learn and temporary pointers to mlflow extensions
cd = CondaDependencies.create(
    conda_packages=["scikit-learn", "matplotlib"],
    pip_packages=["azureml-mlflow", "pandas", "numpy"]
    )

env.python.conda_dependencies = cd

Next, specify a script run configuration that includes the training script, environment and CPU cluster created earlier.

In [6]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=".",
                      script=training_script,
                      compute_target=cpu_cluster,
                      environment=env)

Finally, submit the run. Note that the first instance of the run typically takes longer as the Docker-based environment is created, several minutes. Subsequent runs reuse the image and are faster.

In [7]:
run = exp.submit(src)
run.wait_for_completion(show_output=True)

RunId: RemoteTrain-with-mlflow-sample_1649275840_ec5be499
Web View: https://ml.azure.com/runs/RemoteTrain-with-mlflow-sample_1649275840_ec5be499?wsid=/subscriptions/062bbb35-45d7-40c6-937f-a43ab3667b0f/resourcegroups/rg-dev-geog-eastus/workspaces/mlw-dev-geog-01&tid=5b6f6241-9a57-4be4-8e50-1dfa72e79a57

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Compute======
The run ID for the image build on compute is imgbldrun_3e54a29


Bad pipe message: %s [b'\xb2G\xe6\xff55\xea\xf1&\x16\xae\x8c\xc5\xb0eN\xdc\xcb \xc8\xa8f\xa1\xcb\xd2\xa2z\x0fQ?\xb7\xf2\n\xa5\x1d\x19\x93\xf4\xe8q\xfe\x19\xa0\x1a8ln\x94\x1f\x8c\x9f\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b"\x13u\\\x8cJ9\r\xea\xd7\xa2S\x06 \xaf\x96e\x1b\xfb\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a


Execution Summary
RunId: RemoteTrain-with-mlflow-sample_1649275840_ec5be499
Web View: https://ml.azure.com/runs/RemoteTrain-with-mlflow-sample_1649275840_ec5be499?wsid=/subscriptions/062bbb35-45d7-40c6-937f-a43ab3667b0f/resourcegroups/rg-dev-geog-eastus/workspaces/mlw-dev-geog-01&tid=5b6f6241-9a57-4be4-8e50-1dfa72e79a57



{'runId': 'RemoteTrain-with-mlflow-sample_1649275840_ec5be499',
 'target': 'cpu-cluster',
 'status': 'CancelRequested',
 'startTimeUtc': '2022-04-06T21:05:47.927604Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '02ddba8a-943b-4d7e-839b-ad3acf621190'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_diabetes.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'mlflow-env',
   'version': 'Autosave_2022-04-06T20:10:42Z_0c13da2a',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': F

You can navigate to your Azure ML Workspace at Azure Portal to view the run metrics and artifacts. 

In [ ]:
run

You can also get the metrics and bring them to your local notebook, and view the details of the run.

In [ ]:
run.get_metrics()

In [ ]:
run.get_details()

### Next steps

 * [Deploy the model as a web service](../deploy-model/deploy-model.ipynb)
 * [Learn more about Azure Machine Learning compute options](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets)